In [ ]:
#@title Check availble memory of GPU
# Check that we are using 100% of GPU
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip -q install gputil
!pip -q install psutil
!pip -q install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
!pip install -qq transformers
!pip install -qq nlp

In [ ]:
# imports
from transformers import (
    BertForMaskedLM,
    BertTokenizer,
    BertConfig,
    Trainer,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    LineByLineTextDataset,
)
import torch
from torch.utils.data.dataset import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/My Drive/230T2 MLTS/Colab Notebooks/data/sifted_Speech.txt",
    block_size=128,
)

In [ ]:
pretrain = 'FIN'
if pretrain == 'BERT':
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
elif pretrain =='FIN':
    model = BertForMaskedLM.from_pretrained('/content/drive/My Drive/230T2 MLTS/Colab Notebooks/params/FinBERT-Prime_128MSL-250K')
else:
    config = BertConfig()
    model = BertForMaskedLM(config)
model = model.train()

In [ ]:
training_args = {
    "output_dir": "/content/drive/My Drive/230T2 MLTS/Colab Notebooks/params/"+pretrain,
    "overwrite_output_dir": True,
    "logging_dir": "/content/drive/My Drive/230T2 MLTS/Colab Notebooks/params/logs",
    "learning_rate": 1e-4,
    "do_train": True,
    "do_eval": True,
    "max_steps": 5000,
    # "num_train_epochs":1,
    "warmup_steps": 100,
    "weight_decay": 0.001,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 4,
    "logging_steps": 200,
    "fp16": True,
    "save_steps": 500,
}

training_args = TrainingArguments(**training_args)

In [ ]:
# create the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# train
trainer.train()